In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 30.0 MB/s eta 0:00:00


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md


2023-12-19 08:03:26.841964: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 08:03:26.842036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 08:03:26.843532: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 08:03:26.852228: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 08:03:28.158708: W tensorflow/compiler/tf2

In [ ]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import random
import ipywidgets as widgets
from IPython.display import display
import fitz  # PyMuPDF
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

# Define branch-to-job-role mappings
branch_job_roles = {
    "Artificial Intelligence": ["AI Engineer", "Machine Learning Engineer"],
    "Information Technology": ["Software Developer", "System Reliability Operator"],
    "Computer Science Engineering": ["Web Developer"],
    "Data Science": ["Data Analyst", "Data Scientist"],
    "Cybersecurity": ["Cybersecurity Analyst", "Network Security Engineer", "Security Architect"],
    "Mechanical Engineering": ["Maintainance Engineer", "Automotive Engineer"],
    "Civil Engineering": ["Structural Engineer", "Construction Manager"],
    "MBA - Finance": ["Financial Analyst", "Operations Manager"],
    "MBA - HR": ["Talent Acquisition Manager", "Training and Development Manager"],
    "Electrical Engineering": ["Control System Engineer", "Instrumentation Engineer"],
    "Electronics and Telecommunication Engineering":["Embedded System Engineer", "Telecomm Project Manager"],
}

# Ask the user for their branch
print("Please choose your branch:")
for branch in branch_job_roles.keys():
    print(branch)

user_branch = input("Enter your branch: ").strip()


# Check if the user's branch choice is valid
if user_branch in branch_job_roles:
    branch = user_branch
    job_roles = branch_job_roles[branch]

    print(f"Job roles for {branch}:")
    for i, role in enumerate(job_roles, start=1):
        print(f"{i}. {role}")

    # Ask the user to choose a job role
    user_choice = input("Choose a job role (enter the number): ").strip()

    try:
        user_choice = int(user_choice)
        if 1 <= user_choice <= len(job_roles):
            selected_job_role = job_roles[user_choice - 1]
            print(f"You have selected the job role: {selected_job_role}")
        else:
            print("Invalid choice. Please enter a valid number.")
    except ValueError:
        print("Invalid input. Please enter a number.")
else:
    print("Invalid branch choice. Please choose from the provided options.")

Please choose your branch:
Artificial Intelligence
Information Technology
Computer Science Engineering
Data Science
Cybersecurity
Mechanical Engineering
Civil Engineering
MBA - Finance
MBA - HR
Electrical Engineering
Electronics and Telecommunication Engineering


In [ ]:
# Open the PDF file
pdf_file = "/content/Harsh Dhote data scientist.pdf"
pdf_document = fitz.open(pdf_file)

# Initialize section markers
section_markers = ["Experience", "Education","Skills", "Projects"]
current_section = None
sections = {}

# Initialize NLTK stopwords
stop_words = set(stopwords.words('english'))

# Define a regular expression to match words
word_pattern = re.compile(r'\b\w+\b')

# Iterate through the pages and extract text
for page_number in range(pdf_document.page_count):
    page = pdf_document.load_page(page_number)
    page_text = page.get_text()

    for line in page_text.split('\n'):
        # Check for section markers
        for marker in section_markers:
            if marker.lower() in line.lower():
                current_section = marker
                if current_section not in sections:
                    sections[current_section] = []
                continue

        # If a section has been identified, tokenize and store words
        if current_section:
            # Use regular expression to extract words
            words = word_pattern.findall(line)
            filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
            sections[current_section].extend(filtered_words)


pdf_document.close()
# Create dictionaries to store keywords for each section
section_keywords = {section: words for section, words in sections.items()}

# Print and/or process the words under each section
for section, words in sections.items():
    print(f"Section: {section}")
    print("Keywords:", words)

Section: Experience
Keywords: ['experiences', 'foodiefix', 'ventures', 'data', 'scientist', 'nov', '2021', 'dec', '2021', 'conducted', 'data', 'analysis', 'predictive', 'modeling', 'identify', 'trends', 'patterns', 'utilized', 'data', 'mining', 'feature', 'engineering', 'techniques', 'optimize', 'model', 'performance', 'developed', 'deployed', 'machine', 'learning', 'models', 'real', 'time', 'data', 'driven', 'decision', 'making', 'urbanpulse', 'creations', 'statistical', 'analyst', 'feb', '2022', 'april', '2022', 'performed', 'exploratory', 'data', 'analysis', 'eda', 'regression', 'analysis', 'understand', 'data', 'relationships', 'collaborated', 'classification', 'clustering', 'algorithms', 'customer', 'segmentation', 'utilized', 'natural', 'language', 'processing', 'nlp', 'extract', 'insights', 'unstructured', 'data']
Section: Projects
Keywords: ['projects', 'customer', 'churn', 'prediction', 'github', 'built', 'predictive', 'model', 'identify', 'potential', 'customer', 'churn', 're

In [ ]:
# comparison for creditibility
# matching project with skills
#jaccard similarity
'''x=section_keywords["Projects"]
y=section_keywords["Skills"]
intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
union_cardinality = len(set.union(*[set(x), set(y)]))
credibility=intersection_cardinality/float(union_cardinality)
print(credibility)'''

str1 = " ".join(section_keywords["Projects"])
str2 = " ".join(section_keywords["Skills"])
# Create a CountVectorizer
vectorizer = CountVectorizer().fit_transform([str1, str2])
# Calculate the cosine similarity
cosine_sim = cosine_similarity(vectorizer)
# The cosine similarity score is in cosine_sim[0][1]
similarity_score_credibility= cosine_sim[0][1]

print(f"Credibility : {similarity_score_credibility}")
scaled_similarity_credibility= (similarity_score_credibility + 1) / 2  # Scales to [0, 1]

Credibility : 0.012888481555661677


In [ ]:
job_keywords = {
    "Software Developer": [
        "software","developer","programming","coding","software","engineering","debugging","algorithms","web","development","version","control","testing","problem-solving","software","architecture","code","review","source","code","software","framework","api","integration","software","deployment","agile","methodology","scrum","software","documentation","full-stack","development","unit","testing","object-oriented","programming","continuous","integration","sdlc","software","maintenance","ides","front-end","frameworks","back-end","frameworks","restful","apis","front-end","development","back-end","development","responsive","design","cross","browser","compatibility","ux/ui","design","web","performance","optimization","cms","ajax","user","authentication","server-side","scripting","client-side","scripting","web","security","cloud","computing","api","design","dependency","management","git","code","collaboration","software","quality","assurance","debugging","tools"
        ],
    "Web Developer": [
        "html", "css", "javascript", "front-end", "back-end", "responsive", "design", "user", "experience", "web", "design", "content", "management", "cross-browser", "compatibility", "html5", "css3", "dom", "manipulation", "ajax", "responsive", "frameworks", "web", "templates", "seo", "cms", "ui/ux", "design", "web", "accessibility", "cross-platform", "development", "progressive", "web", "apps", "responsive", "images", "web", "animation", "cross-origin", "resource", "sharing", "user", "authentication", "web", "performance", "optimization", "browser", "developer", "tools", "web", "security", "w3c", "standards", "client-side", "frameworks", "server-side", "frameworks", "restful", "apis", "web", "hosting", "front-end", "build", "tools", "version", "control", "static", "site", "generators", "code", "editors", "web", "development", "best", "practices", "web", "development", "trends", "responsive", "web", "design", "web", "performance", "metrics", "e-commerce", "development", "ajax", "libraries"
        ],
    "Data Scientist": ["data", "analysis", "machine", "learning", "statistics", "data", "visualization", "big", "data", "data", "mining", "python", "r", "data", "modeling", "data", "preprocessing", "predictive", "modeling", "data", "cleaning", "data", "interpretation", "data", "exploration", "data-driven", "decision", "making", "data", "integration", "feature", "engineering", "supervised", "learning", "unsupervised", "learning", "regression", "analysis", "classification", "algorithms", "clustering", "algorithms", "time", "series", "analysis", "data", "pipelines", "data", "wrangling", "data", "transformation", "data", "warehousing", "data", "analytics", "data", "storytelling", "a/b", "testing", "hadoop", "spark", "deep", "learning", "natural", "language", "processing", "computer", "vision", "ensemble", "learning", "model", "evaluation", "cross-validation", "data", "munging", "data", "normalization", "data", "scaling", "data", "indexing", "data", "imputation", "data", "preprocessing", "libraries", "data", "visualization", "tools", "data", "analysis", "software", "data", "reporting", "data", "analytics", "platforms", "data", "exploration", "techniques", "data", "visualization", "techniques", "data", "analytics", "best", "practices", "data", "mining", "tools", "predictive", "modeling", "techniques", "regression", "analysis", "tools", "machine", "learning", "libraries", "feature", "selection", "data", "storytelling", "techniques", "data", "analysis", "libraries", "data-driven", "insights", "data", "interpretation", "tools", "a/b", "testing", "platforms", "big", "data", "technologies", "data", "wrangling", "tools", "data", "transformation", "techniques", "data", "warehousing", "solutions", "time", "series", "analysis", "tools", "clustering", "algorithms", "software", "classification", "algorithms", "libraries" ],
    "Machine Learning Engineer": ["machine", "learning", "deep", "learning", "neural", "networks", "python", "data", "preprocessing", "model", "evaluation", "feature", "engineering", "natural", "language", "processing", "computer", "vision", "data", "modeling", "supervised", "learning", "unsupervised", "learning", "reinforcement", "learning", "nlp", "libraries", "cv", "libraries", "time", "series", "analysis", "ensemble", "learning", "dimensionality", "reduction", "model", "deployment", "hyperparameter", "tuning", "gpu", "acceleration", "automated", "machine", "learning", "transfer", "learning", "data", "augmentation", "model", "interpretation", "model", "selection", "data", "pipelines", "ai", "development", "deep", "learning", "frameworks", "distributed", "computing", "big", "data", "technologies", "algorithm", "development", "tensorflow", "keras", "pytorch", "scikit-learn", "xgboost", "random", "forest", "support", "vector", "machines", "naive", "bayes", "k-means", "clustering", "dbscan", "k-nearest", "neighbors", "recurrent", "neural", "networks", "convolutional", "neural", "networks", "long", "short-term", "memory", "transformer", "models", "word", "embeddings", "image", "recognition", "object", "detection", "natural", "language", "understanding", "machine", "translation", "chatbot", "development", "speech", "recognition", "predictive", "modeling", "model", "evaluation", "metrics", "cross-validation", "techniques", "data", "preprocessing", "tools", "model", "deployment", "platforms", "hyperparameter", "optimization", "gpu", "frameworks", "automated", "machine", "learning", "tools", "transfer", "learning", "libraries", "data", "augmentation", "techniques", "model", "interpretation", "tools", "model", "selection", "techniques", "ai", "development", "frameworks", "deep", "learning", "platforms"
        ],
    "Data Analyst": ["data", "analysis", "data", "visualization", "statistics", "data", "cleaning", "excel", "data", "interpretation", "data", "mining", "pivot", "tables", "data", "reporting", "data", "manipulation", "data", "dashboards", "data", "extraction", "data", "modeling", "data", "exploration", "data", "aggregation", "data", "filtering", "data", "transformation", "data", "validation", "data", "summarization", "data", "migration", "data", "queries", "data", "warehousing", "business", "intelligence", "report", "generation", "dashboard", "tools", "data", "analysis", "software", "data", "visualization", "tools", "statistics", "software", "excel", "functions", "data", "interpretation", "techniques", "data", "mining", "algorithms", "pivot", "table", "functions", "data", "reporting", "tools", "data", "cleaning", "methods", "data", "manipulation", "techniques", "data", "dashboard", "platforms", "data", "extraction", "tools", "data", "modeling", "techniques", "data", "exploration", "methods", "data", "aggregation", "techniques", "data", "filtering", "methods", "data", "transformation", "tools", "data", "validation", "techniques", "data", "summarization", "methods", "data", "migration", "techniques", "data", "query", "tools", "data", "warehousing", "solutions", "business", "intelligence", "platforms", "report", "generation", "tools"

        ],
    "Cybersecurity Analyst": ["cybersecurity", "network", "security", "firewalls", "intrusion", "detection", "security", "policies", "incident", "response", "ethical", "hacking", "vulnerability", "assessment", "penetration", "testing", "threat", "analysis", "security", "protocols", "security", "compliance", "security", "audits", "security", "controls", "security", "architecture", "security", "incident", "management", "malware", "analysis", "security", "awareness", "risk", "management", "security", "assessment", "tools", "threat", "intelligence", "security", "monitoring", "security", "forensics", "security", "software", "firewall", "rules", "cybersecurity", "frameworks", "security", "best", "practices", "security", "assessments", "siem", "ids/ips", "utm", "vulnerability", "scanning", "patch", "management", "incident", "handling", "security", "standards", "network", "monitoring", "security", "incident", "response", "cyber", "threat", "intelligence", "network", "intrusion", "detection", "firewall", "configuration", "ethical", "hacking", "techniques", "vulnerability", "assessment", "tools", "penetration", "testing", "tools", "threat", "analysis", "methods", "security", "protocol", "standards", "security", "compliance", "frameworks", "security", "audit", "tools", "security", "control", "frameworks", "security", "architecture", "principles", "security", "incident", "management", "tools", "malware", "analysis", "techniques", "security", "awareness", "programs", "risk", "management", "strategies", "security", "assessment", "frameworks", "threat", "intelligence", "platforms", "security", "monitoring", "solutions", "security", "forensics", "tools", "security", "software", "platforms", "firewall", "rules", "configuration", "cybersecurity", "framework", "implementation", "security", "best", "practices", "guidelines", "security", "assessment", "methodologies"
        ],

    "AI Engineer": ["artificial", "intelligence", "machine", "learning", "python", "natural", "language", "processing", "deep", "learning", "ai", "algorithms", "data", "preprocessing", "computer", "vision", "neural", "networks", "deep", "learning", "frameworks", "nlp", "libraries", "recommendation", "systems", "ai", "ethics", "reinforcement", "learning", "chatbot", "development", "ai", "deployment", "model", "training", "ai", "model", "evaluation", "ai", "research", "ai", "toolkits", "ai", "platforms", "ai", "data", "pipelines", "image", "recognition", "speech", "recognition", "ai", "applications", "ai", "problem-solving", "ai", "project", "management", "ai", "hardware", "data", "labeling", "ai", "in", "healthcare", "ai", "in", "finance", "ai", "in", "retail", "ai", "in", "education", "ai", "in", "manufacturing", "ai", "in", "autonomous", "vehicles", "ai", "in", "gaming", "ai", "in", "social", "media", "ai", "in", "customer", "service", "ai", "in", "natural", "language", "understanding", "ai", "in", "recommendation", "systems", "ai", "in", "fraud", "detection", "ai", "in", "image", "analysis", "ai", "in", "data", "classification", "ai", "in", "predictive", "maintenance", "ai", "in", "anomaly", "detection", "ai", "in", "language", "translation", "ai", "in", "data", "generation", "ai", "in", "robotics", "ai", "in", "virtual", "reality", "ai", "in", "augmented", "reality", "ai", "in", "speech", "synthesis", "ai", "in", "autonomous", "drones", "ai", "in", "autonomous", "robots", "ai", "in", "autonomous", "machinery", "ai", "in", "autonomous", "systems", "ai", "in", "autonomous", "vehicles", "ai", "in", "game", "development", "ai"],

    "Automotive Engineer": ["automotive", "design", "vehicle", "dynamics", "engine", "mechanics", "vehicle", "safety", "vehicle", "performance", "fuel", "efficiency", "emissions", "control", "crash", "testing", "material", "selection", "automotive", "aerodynamics", "suspension", "systems", "transmission", "systems", "brake", "systems", "steering", "systems", "chassis", "design", "engine", "optimization", "powertrain", "technology", "automotive", "electronics", "autonomous", "vehicles", "connected", "vehicles", "electric", "vehicles", "hybrid", "vehicles", "fuel", "cell", "technology", "emerging", "automotive", "trends", "automotive", "manufacturing", "automotive", "materials", "safety", "regulations", "vehicle", "certification", "vehicle", "testing", "automotive", "prototyping", "vehicle", "modeling", "vehicle", "simulation", "vehicle", "validation", "vehicle", "maintenance", "automotive", "innovation", "automotive", "research", "vehicle", "standards", "vehicle", "systems", "vehicle", "reliability", "vehicle", "ergonomics", "vehicle", "comfort", "vehicle", "aesthetics", "automotive", "cad", "automotive", "cae", "automotive", "manufacturing", "techniques", "automotive", "materials", "selection", "safety", "regulations", "compliance", "vehicle", "certification", "procedures", "vehicle", "testing", "methodologies", "automotive", "prototyping", "methods", "vehicle", "modeling", "techniques", "vehicle", "simulation", "tools", "vehicle", "validation", "procedures", "vehicle", "maintenance", "automotive", "innovation", "automotive", "research", "vehicle", "standards", "vehicle", "systems", "vehicle", "reliability", "vehicle", "ergonomics", "vehicle", "comfort", "vehicle", "aesthetics", "automotive", "cad", "automotive", "cae", "automotive", "manufacturing", "techniques", "automotive", "materials", "selection", "safety", "regulations", "compliance", "vehicle", "certification", "procedures", "vehicle", "testing", "methodologies", "automotive", "prototyping", "methods", "vehicle", "modeling", "techniques", "vehicle", "simulation", "tools", "vehicle", "validation", "procedures", "vehicle", "maintenance", "automotive", "innovation", "automotive", "research", "vehicle", "standards", "vehicle", "systems", "vehicle", "reliability", "vehicle", "ergonomics", "vehicle", "comfort", "vehicle", "aesthetics", "automotive", "cad", "automotive", "cae", "automotive", "manufacturing", "techniques", "automotive", "materials", "selection", "safety", "regulations", "compliance", "vehicle", "certification", "procedures", "vehicle", "testing", "methodologies", "automotive", "prototyping", "methods", "vehicle", "modeling", "techniques", "vehicle", "simulation", "tools", "vehicle", "validation", "procedures", "vehicle", "maintenance", "automotive", "innovation", "automotive", "research", "vehicle", "standards", "vehicle", "systems", "vehicle", "reliability", "vehicle", "ergonomics", "vehicle", "comfort", "vehicle", "aesthetics", "automotive", "cad", "automotive", "cae", "automotive", "manufacturing", "techniques", "automotive", "materials", "selection", "safety", "regulations", "compliance", "vehicle", "certification", "procedures", "vehicle", "testing", "methodologies", "automotive", "prototyping", "methods", "vehicle", "modeling", "techniques"],

    "Structural Engineer": ["structural", "analysis", "building", "design", "construction", "materials", "engineering", "software", "structural", "modeling", "load-bearing", "analysis", "foundation", "design", "structural", "integrity", "structural", "testing", "structural", "inspections", "structural", "rehabilitation", "bridge", "design", "seismic", "analysis", "wind", "analysis", "steel", "structures", "concrete", "structures", "timber", "structures", "masonry", "structures", "retrofitting", "structural", "safety", "structural", "code", "compliance", "finite", "element", "analysis", "structural", "cad", "software", "structural", "engineering", "standards", "building", "information", "modeling", "structural", "stability", "structural", "drawings", "construction", "documentation", "structural", "analysis", "tools", "building", "design", "software", "construction", "materials", "testing", "engineering", "software", "proficiency", "structural", "modeling", "load-bearing", "analysis", "foundation", "design", "structural", "integrity", "structural", "testing", "structural", "inspections", "structural", "rehabilitation", "bridge", "design", "seismic", "analysis", "wind", "analysis", "steel", "structures", "concrete", "structures", "timber", "structures", "masonry", "structures", "retrofitting", "structural", "safety", "structural", "code", "compliance", "finite", "element", "analysis", "structural", "cad", "software", "structural", "engineering", "standards", "building", "information", "modeling", "structural", "stability", "structural", "drawings", "construction", "documentation", "structural", "analysis", "tools", "building", "design", "software", "construction", "materials", "testing", "engineering", "software", "proficiency", "structural", "modeling", "load-bearing", "analysis", "foundation", "design", "structural", "integrity", "structural", "testing", "structural", "inspections", "structural", "rehabilitation", "bridge", "design", "seismic", "analysis", "wind", "analysis", "steel", "structures", "concrete", "structures", "timber", "structures", "masonry", "structures", "retrofitting", "structural", "safety", "structural", "code", "compliance", "finite", "element", "analysis", "structural", "cad", "software", "structural", "engineering", "standards", "building", "information", "modeling", "structural", "stability", "structural", "drawings", "construction", "documentation", "structural", "analysis", "tools", "building", "design", "software", "construction", "materials", "testing", "engineering", "software", "proficiency"

        ],
    "Construction Manager": ["construction", "management", "project", "planning", "budgeting", "site", "supervision", "contract", "management", "quality", "control", "scheduling", "safety", "regulations", "construction", "specifications", "construction", "drawings", "project", "documentation", "construction", "project", "budgeting", "site", "inspections", "subcontractor", "management", "construction", "cost", "estimation", "construction", "scheduling", "regulatory", "compliance", "environmental", "impact", "assessment", "construction", "safety", "risk", "analysis", "construction", "contracts", "construction", "procurement", "stakeholder", "communication", "change", "management", "construction", "progress", "tracking", "resource", "allocation", "conflict", "resolution", "time", "management", "cost", "control", "procurement", "construction", "technology", "lean", "construction", "value", "engineering", "building", "codes", "construction", "standards", "sustainable", "construction", "construction", "management", "software", "project", "management", "software", "construction", "cost", "estimation", "software", "construction", "scheduling", "software", "regulatory", "compliance", "environmental", "impact", "assessment", "software", "construction", "safety", "risk", "analysis", "construction", "contracts", "construction", "procurement", "stakeholder", "communication", "change", "management", "construction", "progress", "tracking", "resource", "allocation", "conflict", "resolution"

        ],
    "Financial Analyst": ["financial", "analysis", "risk", "management", "investment", "analysis", "financial", "modeling", "financial", "reporting", "valuation", "portfolio", "management", "investment", "strategies", "equity", "research", "financial", "markets", "asset", "allocation", "financial", "forecasting", "budgeting", "financial", "software", "data", "analysis", "financial", "statements", "financial", "regulations", "macroeconomics", "microeconomics", "investment", "valuation", "financial", "planning", "business", "finance", "cash", "flow", "analysis", "financial", "modeling", "tools", "investment", "research", "financial", "data", "analysis", "fundamental", "analysis", "technical", "analysis", "market", "research", "financial", "risk", "assessment", "investment", "portfolio", "analysis", "financial", "decision-making", "investment", "decision-making", "capital", "budgeting", "cost", "of", "capital", "financial", "ratios", "financial", "performance", "analysis", "risk", "management", "techniques", "investment", "analysis", "methods", "financial", "modeling", "techniques", "financial", "reporting", "standards", "valuation", "methodologies", "portfolio", "management", "strategies", "investment", "strategies", "implementation", "equity", "research", "principles", "financial", "markets", "analysis", "methods", "asset", "allocation", "techniques", "financial", "forecasting", "methods", "budgeting", "strategies", "financial", "software", "proficiency", "data", "analysis", "techniques", "financial", "statements", "analysis", "financial", "regulations", "adherence", "macroeconomics", "analysis", "microeconomics", "analysis", "investment", "valuation", "techniques", "financial", "planning", "strategies", "business", "finance", "principles", "cash", "flow", "analysis", "methods", "financial", "modeling", "tools", "proficiency", "investment", "research", "methodologies", "financial", "data", "analysis", "techniques", "fundamental", "analysis", "principles", "technical", "analysis", "methods", "market", "research", "strategies", "financial", "risk", "assessment", "techniques", "investment", "portfolio", "analysis", "methods", "financial", "decision-making", "strategies", "investment", "decision-making", "methods", "capital", "budgeting", "techniques", "cost", "of", "capital", "evaluation", "financial", "ratios", "analysis", "financial", "performance", "analysis", "methods"
        ],

    "System Reliability Operator": ["system", "reliability", "uptime", "monitoring", "fault", "detection", "incident", "response", "system", "maintenance", "reliability", "tools", "performance", "optimization", "failover", "planning", "disaster", "recovery", "IT", "infrastructure", "reliability", "standards", "system", "monitoring", "root", "cause", "analysis", "automation", "system", "configuration", "management", "reliability", "metrics", "service", "level", "agreements", "incident", "management", "log", "analysis", "system", "health", "checks", "system", "diagnostics", "reliability", "engineering", "incident", "resolution", "fault", "tolerance", "SLA", "compliance", "system", "recovery", "redundancy", "strategies", "uptime", "optimization", "system", "performance", "metrics", "system", "troubleshooting", "incident", "tracking", "system", "documentation", "system", "architecture", "reliability", "testing", "system", "scalability", "capacity", "planning", "system", "backup", "solutions", "security", "monitoring", "system", "patch", "management", "reliability", "software", "uptime", "improvement", "system", "audit", "availability", "analysis", "incident", "prevention", "reliability", "protocols", "system", "deployment", "system", "reliability", "best", "practices", "uptime", "maintenance", "incident", "remediation", "system", "optimization", "reliability", "audits", "performance", "tuning", "system", "recovery", "procedures", "reliability", "strategy", "development", "uptime", "analysis", "system", "inspection", "incident", "mitigation", "system", "audit", "trails", "SLA", "tracking", "redundancy", "implementation", "uptime", "enhancement", "system", "performance", "analysis", "system", "monitoring", "software", "reliability", "tool", "proficiency", "performance", "optimization", "techniques", "failover", "planning", "strategies", "disaster", "recovery", "procedures", "IT", "infrastructure", "management", "reliability", "standards", "adherence", "system", "monitoring", "methods", "root", "cause", "analysis", "skills", "automation", "tools", "proficiency", "system", "configuration", "management", "expertise", "reliability", "metrics", "analysis", "service", "level", "agreements", "management", "incident", "management", "skills", "log", "analysis", "proficiency", "system", "health", "checks", "procedures", "system", "diagnostics", "proficiency", "reliability", "engineering", "knowledge", "incident", "resolution", "techniques", "fault", "tolerance", "principles", "SLA", "compliance", "tracking", "system", "recovery", "methods", "redundancy", "strategies", "implementation", "uptime", "optimization", "techniques", "system", "performance", "metrics", "analysis", "system", "troubleshooting", "methods", "incident", "tracking", "tools", "utilization", "system", "documentation", "management", "system", "architecture", "knowledge", "reliability", "testing", "procedures", "utilization", "system", "scalability", "principles", "capacity", "planning", "techniques", "system", "backup", "solutions", "management", "security", "monitoring", "techniques", "system", "patch", "management", "procedures", "reliability", "software", "proficiency", "uptime", "improvement", "methods", "system", "audit", "procedures", "utilization", "availability", "analysis", "techniques", "incident", "prevention", "strategies", "reliability", "protocols", "adherence", "system", "deployment", "techniques", "system", "reliability", "best", "practices", "utilization", "uptime", "maintenance", "procedures", "utilization", "incident", "remediation", "methods", "system", "optimization", "techniques", "utilization", "reliability", "audits", "management", "performance", "tuning", "techniques", "system", "recovery", "procedures", "utilization", "reliability", "strategy", "development", "methods", "uptime", "analysis", "techniques", "system", "inspection", "procedures", "utilization", "incident", "mitigation", "strategies", "system", "audit", "trails"
        ],
    "Maintenance Engineer": ["financial", "analysis", "risk", "management", "investment", "analysis", "financial", "modeling", "financial", "reporting", "valuation", "portfolio", "management", "investment", "strategies", "equity", "research", "financial", "markets", "asset", "allocation", "financial", "forecasting", "budgeting", "financial", "software", "data", "analysis", "financial", "statements", "financial", "regulations", "macroeconomics", "microeconomics", "investment", "valuation", "financial", "planning", "business", "finance", "cash", "flow", "analysis", "financial", "modeling", "tools", "investment", "research", "financial", "data", "analysis", "fundamental", "analysis", "technical", "analysis", "market", "research", "financial", "risk", "assessment", "investment", "portfolio", "analysis", "financial", "decision-making", "investment", "decision-making", "capital", "budgeting", "cost", "of", "capital", "financial", "ratios", "financial", "performance", "analysis", "risk", "management", "techniques", "investment", "analysis", "methods", "financial", "modeling", "techniques", "financial", "reporting", "standards", "valuation", "methodologies", "portfolio", "management", "strategies", "investment", "strategies", "implementation", "equity", "research", "principles", "financial", "markets", "analysis", "methods", "asset", "allocation", "techniques", "financial", "forecasting", "methods", "budgeting", "strategies", "financial", "software", "proficiency", "data", "analysis", "techniques", "financial", "statements", "analysis", "financial", "regulations", "adherence", "macroeconomics", "analysis", "microeconomics", "analysis", "investment", "valuation", "techniques", "financial", "planning", "strategies", "business", "finance", "principles", "cash", "flow", "analysis", "methods", "financial", "modeling", "tools", "proficiency", "investment", "research", "methodologies", "financial", "data", "analysis", "techniques", "fundamental", "analysis", "principles", "technical", "analysis", "methods", "market", "research", "strategies", "financial", "risk", "assessment", "techniques", "investment", "portfolio", "analysis", "methods", "financial", "decision-making", "strategies", "investment", "decision-making", "methods", "capital", "budgeting", "techniques", "cost", "of", "capital", "evaluation", "financial", "ratios", "analysis", "financial", "performance", "analysis", "methods" ,"maintenance", "safety", "practices", "maintenance", "metrics", "analysis", "maintenance", "cost", "analysis", "proficiency", "equipment", "calibration", "procedures", "maintenance", "training", "methods", "equipment", "reliability", "knowledge", "maintenance", "compliance", "knowledge", "maintenance", "strategies", "implementation", "maintenance", "optimization", "techniques", "equipment", "installation", "procedures", "maintenance", "support", "methods", "maintenance", "reporting", "practices", "maintenance", "coordination", "skills", "equipment", "upgrades", "knowledge", "maintenance", "supervision", "methods", "maintenance", "records", "management", "maintenance", "tools", "utilization", "equipment", "cleaning", "procedures", "maintenance", "best", "practices", "implementation", "maintenance", "inspections", "procedures", "equipment", "modification", "knowledge", "maintenance", "automation", "tools", "proficiency", "maintenance", "software", "proficiency", "utilization", "equipment", "inspection", "techniques", "utilization", "failure", "analysis", "methods", "utilization", "maintenance", "planning", "procedures", "utilization", "maintenance", "management", "skills", "utilization", "spare", "parts", "inventory", "management", "utilization", "maintenance", "documentation", "practices", "utilization", "root", "cause", "analysis", "proficiency", "utilization", "equipment", "troubleshooting", "techniques", "utilization", "maintenance", "audits", "management", "utilization", "maintenance", "standards", "adherence", "utilization", "equipment", "repair", "knowledge", "utilization", "condition", "monitoring", "techniques", "utilization"
        ],
    "Network Security Engineer": ["network", "security", "firewalls", "intrusion", "detection", "encryption", "security", "protocols", "penetration", "testing", "vulnerability", "assessment", "network", "monitoring", "security", "policies", "incident", "response", "security", "audits", "network", "architecture", "cybersecurity", "access", "control", "security", "solutions", "SIEM", "identity", "and", "access", "management", "risk", "assessment", "security", "compliance", "network", "segmentation", "security", "awareness", "firewall", "configurations", "security", "software", "network", "hardening", "threat", "detection", "security", "assessments", "security", "best", "practices", "network", "protocols", "security", "monitoring", "tools", "incident", "handling", "security", "incident", "reporting", "security", "maintenance", "secure", "communication", "security", "patches", "network", "defense", "security", "testing", "security", "documentation", "security", "implementation", "security", "evaluation", "security", "architecture", "firewalls", "configuration", "intrusion", "detection", "systems", "encryption", "methods", "security", "protocols", "implementation", "penetration", "testing", "techniques", "vulnerability", "assessment", "methods", "network", "monitoring", "tools", "utilization", "security", "policies", "development", "incident", "response", "procedures", "security", "audits", "management", "network", "architecture", "knowledge", "cybersecurity", "principles", "access", "control", "procedures", "security", "solutions", "implementation", "SIEM", "proficiency", "identity", "and", "access", "management", "methods", "risk", "assessment", "techniques", "security", "compliance", "procedures", "network", "segmentation", "strategies", "security", "awareness", "practices", "firewall", "configurations", "proficiency", "security", "software", "utilization", "network", "hardening", "techniques", "threat", "detection", "methods","security", "assessments", "management", "utilization", "best", "practices", "network", "protocols", "knowledge", "monitoring", "tools", "proficiency", "incident", "handling", "procedures", "reporting", "methods", "maintenance", "secure", "communication", "techniques", "patches", "defense", "testing", "documentation", "implementation", "evaluation", "architecture", "firewalls", "configuration", "intrusion", "detection", "systems", "encryption", "methods", "implementation", "penetration", "vulnerability", "assessment", "policies", "development", "audits", "knowledge", "cybersecurity", "principles", "access", "control", "solutions", "SIEM", "identity", "and", "management", "risk", "compliance", "segmentation", "strategies", "awareness", "proficiency", "software", "hardening", "threat", "utilization", "utilization", "proficiency"],
    "Security Architect": ["security", "architecture", "security", "design", "security", "solutions", "security", "frameworks", "cybersecurity", "security", "policies", "security", "assessment", "threat", "modeling", "security", "compliance", "identity", "and", "access", "management", "encryption", "firewalls", "incident", "response", "vulnerability", "assessment", "security", "documentation", "risk", "management", "security", "strategy", "security", "audits", "network", "security", "cloud", "security", "application", "security", "security", "standards", "security", "best", "practices", "security", "protocols", "secure", "coding", "security", "technologies", "penetration", "testing", "SIEM", "security", "governance", "security", "testing", "compliance", "frameworks", "security", "implementation", "security", "awareness", "security", "evaluation", "security", "software", "security", "communication", "architecture", "security", "design", "proficiency", "security", "solutions", "development", "security", "frameworks", "knowledge", "cybersecurity", "principles", "security", "policies", "development", "security", "assessment", "procedures", "threat", "modeling", "techniques", "security", "compliance", "procedures", "identity", "and", "access", "management", "knowledge", "encryption", "methods", "firewalls", "configuration", "incident", "response", "procedures", "vulnerability", "assessment", "methods", "security", "documentation", "practices", "risk", "management", "principles", "security", "strategy", "development", "security", "audits", "management", "network", "security", "expertise", "cloud", "security", "knowledge", "application", "security", "principles", "security", "standards", "adherence", "security", "best", "practices", "utilization", "security", "protocols", "implementation", "secure", "coding", "techniques", "security", "technologies", "knowledge", "penetration", "testing", "procedures", "SIEM", "proficiency", "security", "governance", "practices", "security", "testing", "techniques", "compliance", "frameworks", "utilization", "security", "implementation", "methods", "security", "awareness", "practices", "security", "evaluation", "techniques", "security", "software", "proficiency", "security", "communication", "knowledge", "architecture", "security", "design", "proficiency", "utilization", "security", "solutions", "development", "utilization", "security", "frameworks", "knowledge", "utilization", "cybersecurity", "principles", "utilization", "security", "policies", "development", "utilization", "security", "assessment", "procedures", "utilization", "threat", "modeling", "techniques", "utilization", "security", "compliance", "procedures", "utilization", "identity", "and", "access", "management", "knowledge", "utilization", "encryption", "methods", "utilization", "firewalls", "configuration", "utilization", "incident", "response", "procedures", "utilization", "vulnerability", "assessment", "methods", "utilization", "security", "documentation", "practices", "utilization", "risk", "management", "principles", "utilization", "security", "strategy", "development", "utilization", "security", "audits", "management", "utilization", "network", "security", "expertise", "utilization", "cloud", "security", "knowledge", "utilization", "application", "security", "principles", "utilization", "security", "standards", "adherence", "utilization", "security", "best", "practices", "utilization", "utilization", "security", "protocols", "implementation", "utilization", "secure", "coding", "techniques", "utilization", "security", "technologies", "knowledge", "utilization", "penetration", "testing", "procedures", "utilization", "SIEM", "proficiency", "utilization", "security", "governance", "practices", "utilization", "security", "testing", "techniques", "utilization", "compliance", "frameworks", "utilization", "utilization", "security", "implementation", "methods", "utilization", "security", "awareness", "practices", "utilization", "security", "evaluation", "techniques", "utilization", "security", "software"],
    "Operations Manager": ["operations", "management", "team", "leadership", "workflow", "optimization", "strategic", "planning", "resource", "allocation", "project", "management","process", "improvement", "budget", "management", "performance", "metrics", "quality", "control", "business", "operations", "staff", "development", "cross-functional","collaboration", "problem-solving", "efficiency", "improvement", "communication", "skills", "operations", "analysis", "risk", "management", "supply", "chain", "management","vendor", "relationships", "cost", "reduction", "operations", "strategy", "inventory", "management", "production", "planning", "logistics", "scheduling", "customer", "service","data", "analysis", "change", "management", "continuous", "improvement", "leadership", "development", "decision-making", "workforce", "management", "operations", "research","KPIs", "workplace", "safety", "process", "reengineering", "strategic", "partnerships", "operations", "software", "project", "implementation", "financial", "analysis","team", "building", "operations", "budgeting", "workflow", "optimization", "techniques", "strategic", "planning", "proficiency", "resource", "allocation", "strategies","project", "management", "skills", "process", "improvement", "methods", "budget", "management", "proficiency", "performance", "metrics", "analysis", "quality", "control", "practices","business", "operations", "knowledge", "staff", "development", "techniques", "cross-functional", "collaboration", "skills", "problem-solving", "proficiency", "efficiency", "improvement", "strategies","communication", "skills", "proficiency", "operations", "analysis", "techniques", "risk", "management", "practices", "supply", "chain", "management", "knowledge", "vendor", "relationships", "management","cost", "reduction", "strategies", "operations", "strategy", "development", "inventory", "management", "proficiency", "production", "planning", "techniques", "logistics", "management","scheduling", "procedures", "customer", "service", "knowledge", "data", "analysis", "methods", "change", "management", "techniques", "continuous", "improvement", "practices","leadership", "development", "methods", "decision-making", "skills", "workforce", "management", "proficiency", "operations", "research", "knowledge", "KPIs", "tracking", "workplace","safety", "practices", "process", "reengineering", "techniques", "strategic", "partnerships", "management", "operations", "software", "utilization", "project", "implementation", "methods","financial", "analysis", "proficiency", "team", "building", "skills", "operations", "budgeting", "practices", "workflow", "optimization", "techniques", "utilization", "strategic", "planning", "proficiency","utilization", "resource", "allocation", "strategies", "utilization", "project", "management", "skills", "utilization", "process", "improvement", "methods", "utilization", "budget", "management","proficiency", "utilization", "performance", "metrics", "analysis", "utilization", "quality", "control", "practices", "utilization", "business", "operations", "knowledge", "utilization","staff", "development", "techniques", "utilization", "cross-functional", "collaboration", "skills", "utilization", "problem-solving", "proficiency", "utilization", "efficiency", "improvement","strategies", "utilization", "communication", "skills", "proficiency", "utilization", "operations", "analysis", "techniques", "utilization", "risk", "management", "practices", "utilization","supply", "chain", "management", "knowledge", "utilization", "vendor", "relationships", "management", "utilization", "cost", "reduction", "strategies", "utilization", "operations", "strategy","development", "utilization", "inventory", "management", "proficiency", "utilization", "production", "planning", "techniques", "utilization", "logistics", "management", "utilization","scheduling", "procedures", "utilization", "customer", "service", "knowledge", "utilization", "data", "analysis", "methods", "utilization", "change"],

    "Training and Development Manager": ["training", "and", "development", "employee", "learning", "performance", "improvement", "training", "programs", "adult", "learning", "principles", "training", "needs", "analysis","instructional", "design", "e-learning", "training", "delivery", "evaluation", "of", "training", "effectiveness", "leadership", "development", "organizational", "development","training", "budgeting", "learning", "management", "systems", "staff", "development", "talent", "development", "curriculum", "design", "training", "assessment", "training","content", "creation", "training", "materials", "employee", "skills", "development", "soft", "skills", "training", "on-the-job", "training", "training", "ROI","analysis", "e-learning", "platforms", "employee", "engagement", "training", "technologies", "assessment", "tools", "training", "software", "training", "documentation","employee", "development", "planning", "performance", "management", "systems", "employee", "feedback", "mechanisms", "training", "and", "development", "strategies", "training","program", "management", "adult", "learning", "principles", "application", "training", "needs", "analysis", "methods", "instructional", "design", "principles","e-learning", "platforms", "utilization", "training", "delivery", "methods", "evaluation", "of", "training", "effectiveness", "techniques", "leadership", "development","strategies", "organizational", "development", "principles", "training", "budgeting", "techniques", "learning", "management", "systems", "utilization", "staff", "development",
        "methods", "talent", "development", "strategies", "curriculum", "design", "techniques", "training", "assessment", "methods", "training", "content", "creation","proficiency", "training", "materials", "development", "employee", "skills", "development", "methods", "soft", "skills", "training", "techniques", "on-the-job","training", "procedures", "training", "ROI", "analysis", "methods", "e-learning", "platforms", "utilization", "proficiency", "employee", "engagement", "strategies", "training","technologies", "utilization", "assessment", "tools", "utilization", "training", "software", "proficiency", "training", "documentation", "management", "employee",
        "development", "planning", "strategies", "performance", "management", "systems", "utilization", "employee", "feedback", "mechanisms", "utilization", "training", "and","development", "strategies", "implementation", "training", "program", "management", "proficiency", "adult", "learning", "principles", "application","proficiency", "training", "needs", "analysis", "methods", "proficiency", "instructional", "design", "principles", "proficiency", "e-learning", "platforms", "utilization","proficiency", "training", "delivery", "methods", "proficiency", "evaluation", "of", "training", "effectiveness", "techniques", "proficiency", "leadership", "development",
        "strategies", "proficiency", "organizational", "development", "principles", "proficiency", "training", "budgeting", "techniques", "proficiency", "learning", "management","systems", "utilization", "proficiency", "staff", "development", "methods", "proficiency", "talent", "development", "strategies", "proficiency", "curriculum", "design","techniques", "proficiency", "training", "assessment", "methods", "proficiency", "training", "content", "creation", "proficiency", "utilization", "training", "materials","development", "utilization", "employee", "skills", "development", "methods", "utilization", "soft", "skills", "training", "techniques", "utilization", "on-the-job",
        "training", "procedures", "utilization", "training", "ROI", "analysis", "methods", "utilization", "e-learning", "platforms", "utilization", "proficiency","utilization", "employee", "engagement", "strategies", "utilization", "training", "technologies", "utilization", "proficiency", "utilization", "assessment", "tools","utilization", "proficiency"],
    "Talent Acquisition Manager": ["talent", "acquisition", "recruitment", "sourcing", "interviewing", "candidate", "assessment", "employer", "branding","recruitment", "strategies", "job", "advertising", "applicant", "tracking", "systems", "interview", "techniques", "onboarding","recruitment", "metrics", "diversity", "hiring", "staffing", "agencies", "talent", "pipelines", "recruitment", "compliance", "HR", "technology","offer", "negotiation", "candidate", "experience", "talent", "acquisition", "software", "recruitment", "analytics", "interviewing", "skills","employer", "branding", "strategies", "recruitment", "strategies", "implementation", "job", "advertising", "techniques", "applicant", "tracking", "systems", "utilization","interview", "techniques", "proficiency", "onboarding", "processes", "recruitment", "metrics", "analysis", "diversity", "hiring", "strategies", "staffing", "agencies", "collaboration","talent", "pipelines", "development", "recruitment", "compliance", "adherence", "HR", "technology", "utilization", "offer", "negotiation", "skills", "candidate", "experience", "enhancement","talent", "acquisition", "software", "proficiency", "recruitment", "analytics", "utilization", "interviewing", "skills", "proficiency", "employer", "branding", "strategies", "implementation","recruitment", "strategies", "implementation", "proficiency", "job", "advertising", "techniques", "proficiency", "applicant", "tracking", "systems", "utilization", "proficiency","interview", "techniques", "proficiency", "utilization", "onboarding", "processes", "utilization", "recruitment", "metrics", "analysis", "proficiency", "diversity", "hiring", "strategies", "utilization","staffing", "agencies", "collaboration", "proficiency", "talent", "pipelines", "development", "utilization", "recruitment", "compliance", "adherence", "proficiency", "HR", "technology", "utilization", "proficiency","offer", "negotiation", "skills", "utilization", "candidate", "experience", "enhancement", "utilization", "talent", "acquisition", "software", "proficiency", "utilization", "recruitment", "analytics", "utilization","interviewing", "skills", "proficiency", "utilization"
        ],
    "Control Systems Engineer": ["control", "systems", "automation", "PLC", "programming", "SCADA", "systems", "process", "control", "instrumentation", "system", "integration", "control", "system", "design", "control", "algorithms", "PID", "control", "HMI", "design", "industrial", "networks", "sensors", "actuators", "control", "system", "troubleshooting", "PLC", "programming", "languages", "SCADA", "systems", "configuration", "process", "control", "techniques", "instrumentation", "principles", "system", "integration", "methods", "control", "system", "design", "principles", "control", "algorithms", "development", "PID", "control", "principles", "HMI", "design", "proficiency", "industrial", "networks", "management", "sensors", "utilization", "actuators", "operation", "control", "system", "troubleshooting", "methods", "PLC", "programming", "languages", "proficiency", "SCADA", "systems", "configuration", "proficiency", "process", "control", "techniques", "proficiency", "instrumentation", "principles", "utilization", "system", "integration", "methods", "proficiency", "control", "system", "design", "principles", "utilization", "control", "algorithms", "development", "proficiency", "PID", "control", "principles", "utilization", "HMI", "design", "proficiency", "utilization", "industrial", "networks", "management", "proficiency", "sensors", "utilization", "proficiency", "actuators", "operation", "proficiency", "control", "system", "troubleshooting", "methods", "utilization"
        ],
    "Instrumentation Engineer": [
        "instrumentation", "sensors", "measurement", "devices", "calibration", "control", "systems", "data", "acquisition", "signal", "processing", "instrumentation", "design", "sensor", "selection", "measurement", "accuracy", "industrial", "automation", "PLC", "programming", "HMI", "interface", "instrumentation", "troubleshooting", "sensors", "calibration", "data", "analysis", "sensor", "technologies", "measurement", "principles", "industrial", "control", "systems", "PLC", "programming", "proficiency", "HMI", "interface", "design", "instrumentation", "troubleshooting", "methods", "sensors", "calibration", "techniques", "data", "analysis", "proficiency", "sensor", "technologies", "utilization", "measurement", "principles", "application", "industrial", "control", "systems", "utilization"
        ],
    "Embedded Systems Engineer": [
       "embedded", "systems", "microcontrollers", "firmware", "development", "real-time", "operating", "systems", "embedded", "C/C++", "programming", "electronics", "schematic", "design", "PCB", "layout", "peripheral", "interfacing", "debugging", "sensor", "integration", "IoT", "devices", "RTOS", "embedded", "software", "low-level", "programming", "hardware/software", "integration", "microcontroller", "firmware", "debugging", "electronics", "knowledge", "schematic", "design", "proficiency", "PCB", "layout", "design", "peripheral", "interfacing", "methods", "debugging", "techniques", "sensor", "integration", "proficiency", "IoT", "devices", "development", "RTOS", "development", "methods", "embedded", "software", "design", "proficiency", "low-level", "programming", "skills", "hardware/software", "integration", "proficiency", "microcontroller", "programming", "expertise", "firmware", "debugging", "techniques", "electronics", "knowledge", "proficiency", "schematic", "design", "proficiency", "utilization", "PCB", "layout", "design", "utilization", "peripheral", "interfacing", "methods", "utilization", "debugging", "techniques", "utilization", "sensor", "integration", "proficiency", "utilization", "IoT", "devices", "development", "utilization", "RTOS", "development", "methods", "utilization", "embedded", "software", "design", "proficiency", "utilization", "low-level", "programming", "skills", "utilization", "hardware/software", "integration", "proficiency", "utilization", "microcontroller", "programming", "expertise", "utilization", "firmware", "debugging", "techniques", "utilization"
        ],
    "Telecom Project Manager": ["telecom", "project", "management", "telecommunications", "network", "infrastructure", "telecom", "projects", "budgeting", "stakeholder", "communication", "vendor", "management", "project", "planning", "resource", "allocation", "risk", "management", "quality", "assurance", "telecom", "technologies", "telecom", "regulations", "telecom", "network", "design", "fiber", "optics", "wireless", "networks", "VoIP", "telecom", "software", "telecom", "project", "lifecycle", "telecom", "project", "documentation", "network", "optimization", "telecom", "project", "delivery", "telecom", "project", "budgeting", "stakeholder", "communication", "strategies", "vendor", "management", "techniques", "project", "planning", "methods", "resource", "allocation", "strategies", "risk", "management", "techniques", "quality", "assurance", "procedures", "telecom", "technologies", "utilization", "telecom", "regulations", "compliance", "telecom", "network", "design", "proficiency", "fiber", "optics", "knowledge", "wireless", "networks", "management", "VoIP", "implementation", "telecom", "software", "utilization", "telecom", "project", "lifecycle", "management", "telecom", "project", "documentation", "management", "network", "optimization", "strategies", "telecom", "project", "delivery", "methods", "telecom", "project", "budgeting", "proficiency", "stakeholder", "communication", "strategies", "utilization", "vendor", "management", "techniques", "utilization", "project", "planning", "methods", "utilization", "resource", "allocation", "strategies", "utilization", "risk", "management", "techniques", "utilization", "quality", "assurance", "procedures", "utilization", "telecom", "technologies", "utilization", "proficiency", "telecom", "regulations", "compliance", "proficiency", "telecom", "network", "design", "proficiency", "utilization", "fiber", "optics", "knowledge", "proficiency", "wireless", "networks", "management", "proficiency", "VoIP", "implementation", "proficiency", "telecom", "software", "utilization", "proficiency", "telecom", "project", "lifecycle", "management", "proficiency", "telecom", "project", "documentation", "management", "proficiency", "network", "optimization", "strategies", "utilization", "telecom", "project", "delivery", "methods", "utilization"]
}

In [ ]:
# comparison for relevency
# matching job role with skills


'''x=job_keywords[selected_job_role]
y=section_keywords["Skills"]
intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
union_cardinality = len(set.union(*[set(x), set(y)]))
print(intersection_cardinality/float(union_cardinality))
print(job_keywords[selected_job_role])'''

keywords_str3 = " ".join(job_keywords[selected_job_role])
keywords_str4 = " ".join(section_keywords["Skills"])
# Create a CountVectorizer
vectorizer = CountVectorizer().fit_transform([keywords_str3,keywords_str4])
# Calculate the cosine similarity
cosine_sim = cosine_similarity(vectorizer)
# The cosine similarity score is in cosine_sim[0][1]
similarity_score_relevancy = cosine_sim[0][1]

print(f"Relevancy: {similarity_score_relevancy}")
scaled_similarity_relevancy= (similarity_score_relevancy + 1) / 2  # Scales to [0, 1]

Relevancy: 0.8709578388786349


In [ ]:
# Load the dataset
df = pd.read_csv('/content/Ques Answer dataset - Sheet1 (2).csv')

chosen_job_role = selected_job_role

print("Questions for the job role of",chosen_job_role)

class QuestionAsker:
    def __init__(self,chosen_job_role):
        self.user_responses = []
        self.question_index = 0
        self.total_score = 0
        self.chosen_job_role = chosen_job_role
        self.nlp = spacy.load('en_core_web_md')
        self.question_answer_score = None  # Initialize the score variable

    def map_similarity_to_score(self, similarity):
        return round(similarity * 10, 2)

    def get_unique_questions(self, job_role):
        questions = df[df['Job Role'] == job_role]['Question'].unique()
        return random.sample(list(questions), min(len(questions), 5))  # Asking only 5 questions

    def display_next_question(self, b):
        if self.question_index < len(self.sampled_questions):
            question = self.sampled_questions[self.question_index]
            answer = df[(df['Question'] == question) & (df['Job Role'] == self.chosen_job_role)]['Answer'].iloc[0]

            print(f"Q: {question}\n")
            print("Please provide your answer:")

            user_response = widgets.Text(
                value="",
                description="Your Answer:",
            )
            display(user_response)

            next_button = widgets.Button(description="Next")
            display(next_button)

            def handle_next_button_click(b):
                user_answer = user_response.value
                self.user_responses.append(user_answer)

                doc1 = self.nlp(user_answer)
                doc2 = self.nlp(answer)
                similarity = cosine_similarity([doc1.vector], [doc2.vector])[0][0]
                score = self.map_similarity_to_score(similarity)
                self.total_score += score
                print(f"Score for Question: {score}/10")
                print(f"Your Answer: {user_answer}\n")

                user_response.close()
                next_button.close()
                self.question_index += 1
                self.display_next_question(None)

            next_button.on_click(handle_next_button_click)
        else:
            average_score_per_question = self.total_score / len(self.sampled_questions)
            total_average_score_out_of_50 = round((average_score_per_question / 10) * 50, 2)  # Total score out of 50

            # Store the total average score in the 'question-answer-score' variable
            self.question_answer_score = total_average_score_out_of_50
            # print(f"Total Average Score Out of 50: {total_average_score_out_of_50}/50")

question_asker = QuestionAsker(chosen_job_role)

question_asker.sampled_questions = question_asker.get_unique_questions(question_asker.chosen_job_role)
question_asker.display_next_question(None)

Questions for the job role of Data Scientist
Q: Can you explain the key steps involved in the data analysis process?

Please provide your answer:


Text(value='', description='Your Answer:')

Button(description='Next', style=ButtonStyle())

Score for Question: -0.34/10
Your Answer: dont know

Q: How do you assess the importance of different variables in a machine learning model?

Please provide your answer:


Text(value='', description='Your Answer:')

...

Button(description='Next', style=ButtonStyle())

Score for Question: 0.0/10
Your Answer: odf

Q: Explain the difference between batch gradient descent and stochastic gradient descent in training machine learning models.

Please provide your answer:


Text(value='', description='Your Answer:')

Button(description='Next', style=ButtonStyle())

Score for Question: -0.02/10
Your Answer: jv



In [ ]:
# Print the total average score from the QuestionAsker
que_total_score = question_asker.question_answer_score
print(f"Total Score Out of 50:", que_total_score)

Total Score Out of 50: 8.7


In [ ]:
#average total score percentile
a=scaled_similarity_relevancy*40
b=scaled_similarity_credibility*30
c=que_total_score*30/50
print(a+b+c)

57.83248400090762


In [ ]:
#feedback
#for relevency score
c = similarity_score_credibility
if c<=0.1:
  feedback_c = "Ensure that the skills you list in your resume align closely with the projects you've worked on."
elif 0.1<c<=0.2:
  feedback_c= " Be more descriptive about the projects you've undertaken. Elaborate on the role you played and the outcomes achieved. "
elif 0.2<c<=0.3:
  feedback_c= "Tailor your skills section to emphasize those that directly relate to your projects."
elif 0.3<c<=0.4:
  feedback_c= "In your project descriptions, use action verbs to clearly indicate the skills you utilized."
elif 0.4<c<=0.5:
  feedback_c= "Even if your projects aren't an exact match for your desired job, emphasize transferable skills."
elif 0.5<c<=0.6:
  feedback_c= "Don't use a one-size-fits-all approach. Tailor your resume for each job application, emphasizing the skills and projects that are most relevant to the specific role."
elif c>0.6:
  feedback_c= "Good job!! Your resume looks creditable enough!"

r= similarity_score_relevancy
if r<=0.1:
  feedback_r = "Ensure that the skills listed in your resume closely match the skills required for the job role."
elif 0.1<r<=0.2:
  feedback_r= " Prioritize and emphasize the skills that directly align with the job description."
elif 0.2<r<=0.3:
  feedback_r= " Incorporate keywords from the job description into your resume. This can improve your chances of passing through automated applicant tracking systems (ATS)."
elif 0.3<r<=0.4:
  feedback_r= " If there are gaps between the skills you possess and those required for the job, consider upskilling or obtaining relevant certifications to bridge the gap."
elif 0.4<r<=0.5:
  feedback_r= " Highlight your ability to learn new skills quickly."
elif 0.5<r<=0.6:
  feedback_r= "Customizing your resume to match the skills required for the job role is crucial in increasing your chances of being considered for the position."
elif r>0.6:
  feedback_r= "Good job!! Your resume looks relevant enough with the Job Role you are applying for!!"

q=que_total_score
if q<=10:
  feedback_q = "he answer is completely inadequate, providing no relevant information about skills and technical specifications. It is incoherent and entirely misses the mark in addressing the question."
elif 10<q<=15:
  feedback_q= " The answer is very weak, offering almost no relevant information about skills and technical specifications. It's confusing and lacks substance, requiring a substantial overhaul.  "
elif 15<q<=20:
  feedback_q= " The answer is weak, providing minimal and vague information about skills and technical specifications. It lacks clarity and specific examples, indicating a significant need for improvement."
elif 20<q<=25:
  feedback_q= " The answer is subpar, offering limited and basic information about skills and technical specifications. It's somewhat relevant but requires significant improvement to be competitive."
elif 25<q<=30:
  feedback_q= " The answer is neither impressive nor underwhelming. It adequately addresses the question and includes basic information about skills and technical specifications but lacks depth or specific examples."
elif 30<r<=35:
  feedback_q= "The answer is above average, adequately addressing the question and highlighting relevant skills and technical specifications. It's clear and concise but could benefit from more depth or specific examples."
elif q>35:
  feedback_q= "The answer is well-structured, addressing most relevant skills and technical specifications with a good level of understanding. It includes specific examples but may have minor areas for improvement.!"

print(feedback_c)
print(feedback_r)
print(feedback_q)


Ensure that the skills you list in your resume align closely with the projects you've worked on.
Good job!! Your resume looks relevant enough with the Job Role you are applying for!!
he answer is completely inadequate, providing no relevant information about skills and technical specifications. It is incoherent and entirely misses the mark in addressing the question.
